In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm

def load_and_preprocess_dataset(file_path):
    try:
        data = pd.read_csv(file_path)
        data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])

        data['year'] = data['trans_date_trans_time'].dt.year
        data['month'] = data['trans_date_trans_time'].dt.month
        data['day'] = data['trans_date_trans_time'].dt.day
        data['hour'] = data['trans_date_trans_time'].dt.hour
        data['minute'] = data['trans_date_trans_time'].dt.minute


        data = data.drop('trans_date_trans_time', axis=1)


        data.dropna(subset=['is_fraud'], inplace=True)


        X = data.drop('is_fraud', axis=1)
        y = data['is_fraud']

        numeric_columns = X.select_dtypes(include=['number'])
        X[numeric_columns.columns] = numeric_columns.fillna(numeric_columns.mean())


        numeric_features = X.select_dtypes(include=['number']).columns
        categorical_features = X.select_dtypes(exclude=['number']).columns

        numeric_transformer = StandardScaler()
        categorical_transformer = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)
            ])


        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier())
        ])

        return pipeline, X, y
    except Exception as e:
        print(f"Error loading and preprocessing dataset: {str(e)}")
        return None, None, None

def train_and_evaluate_model(pipeline, X, y):
    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print("Training the model...")

        with tqdm(total=len(X_train)) as pbar:
            pipeline.named_steps['classifier'].n_jobs = -1
            pipeline.fit(X_train, y_train)

            pbar.update(len(X_train))

        print("Evaluating the model...")
        y_pred = pipeline.predict(X_test)


        print("Classification Report:")
        print(classification_report(y_test, y_pred))

        print("Confusion Matrix:")
        print(confusion_matrix(y_test, y_pred))

        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy * 100:.2f}%")
    except Exception as e:
        print(f"Error training and evaluating the model: {str(e)}")

file_path = "fraudTrain.csv"

pipeline, X, y = load_and_preprocess_dataset(file_path)

if pipeline is not None:

    train_and_evaluate_model(pipeline, X, y)


Training the model...


  0%|          | 0/276076 [00:00<?, ?it/s]